In [183]:
from pykrx import stock
import pandas as pd
import numpy as np
import string
import price 
import imp
imp.reload(price)

C:\Users\0327c\AppData\Local\Temp\ipykernel_2596\399083486.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


<module 'price' from 'c:\\수익왕 김퀀트\\price.py'>

#### pykrx에서 6월 30일 기준 수정종가데이터 추출 (2011-2022)

기준일의 코스피와 코스닥의 종목코드 추출

In [123]:
codes_kospi = stock.get_market_ticker_list('20110630', market='KOSPI') # 코스피, 코스닥 데이터의 종목코드 데이터
codes_kosdaq = stock.get_market_ticker_list('20110630', market='KOSDAQ')
codes = []
for i in range(len(codes_kospi)):
    codes.append(codes_kospi[i])
for i in range(len(codes_kosdaq)):
    codes.append(codes_kosdaq[i])
len(codes)

1959

종목코드 + 종목명 데이터프레임, 종목코드 인덱스화

In [177]:
corp = []
for code in codes:
    name = stock.get_market_ticker_name(code) # 코스피, 코스닥 데이터의 종목명 데이터
    corp.append([code, name])
df1 = pd.DataFrame(data=corp, columns=['티커', '종목명'])
df1 = df1.set_index('티커') # 종목코드 인덱스화 
df1

,종목명
티커,
138930,BNK금융지주
001460,BYC
001465,BYC우
001040,CJ
079160,CJ CGV
...,...
024060,흥구석유
010240,흥국
037440,희림


In [178]:
list = []
for i in range(len(df1)):
    if '스팩' in df1['종목명'][i] or df1.index[i][-1] != '0':
        list.append(df1.index[i])
df1 = df1.drop(list, axis=0)
df1

,종목명
티커,
138930,BNK금융지주
001460,BYC
001040,CJ
079160,CJ CGV
011150,CJ씨푸드
...,...
084110,휴온스글로벌
024060,흥구석유
010240,흥국


기준일 코스피, 코스닥의 종가, 상장주식수만 추출

In [179]:
df_kospi = stock.get_market_cap_by_ticker(date='20110630', market='KOSPI')  # 코스피 종가 
df_kosdaq = stock.get_market_cap_by_ticker(date='20110630', market='KOSDAQ') # 코스닥 종가
df = pd.concat([df_kospi, df_kosdaq])
df = df.drop(['시가총액','거래대금'], axis=1)

기준일 1년 후 코스피, 코스닥의 종가, 상장주식수만 추출

In [180]:
df_kospi_1= stock.get_market_cap_by_ticker(date='20120629', market='KOSPI')  # 1년 후 코스피 종가
df_kosdaq_1 = stock.get_market_cap_by_ticker(date='20120629', market='KOSDAQ')  
df_1 = pd.concat([df_kospi_1, df_kosdaq_1])
df_1 = df_1.drop(['시가총액', '거래량', '거래대금'], axis=1)
df_1

,종가,상장주식수
티커,,
005930,1201000,147299337
005380,232500,220276479
005490,363500,87186835
000270,74700,405363347
012330,274500,97343863
...,...,...
004325,2080,1400000
013780,430,4742410
032685,11550,142857


df1, df, df_1 데이터프레임 병합

In [181]:
df_total = pd.merge(df, df_1, how='left', on=['티커'])
df_total = pd.merge(df1, df_total, how='left', on=['티커'])
df_total.columns = ['종목명', '종가', '거래량', '상장주식수', '1년후종가', '1년후상장주식수']
df_total['상장주식수변동'] = df_total['1년후상장주식수'] - df_total['상장주식수']
df_total

,종목명,종가,거래량,상장주식수,1년후종가,1년후상장주식수,상장주식수변동
티커,,,,,,,
138930,BNK금융지주,15100,821667,193379899,12650.0,193379899.0,0.0
001460,BYC,153000,227,624615,167500.0,624615.0,0.0
001040,CJ,72700,347648,28982589,79900.0,29004388.0,21799.0
079160,CJ CGV,29650,89692,20617458,25950.0,20617458.0,0.0
011150,CJ씨푸드,2435,169986,35930773,2410.0,35930773.0,0.0
...,...,...,...,...,...,...,...
084110,휴온스글로벌,5760,32072,9000000,11450.0,9000000.0,0.0
024060,흥구석유,1760,23650,15000000,1685.0,15000000.0,0.0
010240,흥국,5040,22595,6161348,4180.0,6161348.0,0.0


위 내용으로 만든 종가 데이터 뽑는 함수
* price.price_func('기준일', '1년후')

In [185]:
price.price_func('20110630', '20110629')

,종목명,종가,거래량,상장주식수,1년후종가,1년후상장주식수,상장주식수변동
티커,,,,,,,
138930,BNK금융지주,15100,821667,193379899,15100.0,193379899.0,0.0
001460,BYC,153000,227,624615,152500.0,624615.0,0.0
001040,CJ,72700,347648,28982589,73400.0,28982589.0,0.0
079160,CJ CGV,29650,89692,20617458,29150.0,20617458.0,0.0
011150,CJ씨푸드,2435,169986,35930773,2425.0,35930773.0,0.0
...,...,...,...,...,...,...,...
084110,휴온스글로벌,5760,32072,9000000,5790.0,9000000.0,0.0
024060,흥구석유,1760,23650,15000000,1755.0,15000000.0,0.0
010240,흥국,5040,22595,6161348,4995.0,6161348.0,0.0
